In [1]:
import psycopg2

import pandas as pd
import numpy as np
import mariadb
# import dask.dataframe as dd
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc
from datetime import datetime
from dateutil import parser

In [2]:
hydroponics_df_time_filtered = pd.read_csv('input/hydroponics_samples_.csv')

In [3]:
hydroponics_df_time_filtered

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,513837,513837,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,boron,0.026,ppm
1,513838,513838,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,copper,0.014,ppm
2,513825,513825,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,electrical_conductivity,0.330,mS/cm
3,513836,513836,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,zinc,0.054,ppm
4,513824,513824,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,ph,9.380,NaN
...,...,...,...,...,...,...,...,...
707498,540791,540791,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,electrical_conductivity,2.160,mS/cm
707499,550819,550819,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,boron,0.092,ppm
707500,550818,550818,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,zinc,0.210,ppm
707501,550822,550822,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,sodium,28.500,ppm


In [4]:
hydroponics_df_time_filtered['analysis_name'] = [ str(i).split(",") for i in hydroponics_df_time_filtered['analysis_name'] ]

In [5]:
hydroponics_df_time_filtered= hydroponics_df_time_filtered.explode("analysis_name")

In [6]:
hydroponics_df_time_filtered

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,513837,513837,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,boron,0.026,ppm
1,513838,513838,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,copper,0.014,ppm
2,513825,513825,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,electrical_conductivity,0.330,mS/cm
3,513836,513836,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,zinc,0.054,ppm
4,513824,513824,CS032HY0239,2023-03-28 16:33:43.000,Substrate Analysis (Pumice) 1:1.5 Volume Extract,ph,9.380,NaN
...,...,...,...,...,...,...,...,...
707498,540791,540791,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,electrical_conductivity,2.160,mS/cm
707499,550819,550819,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,boron,0.092,ppm
707500,550818,550818,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,zinc,0.210,ppm
707501,550822,550822,CH031HY0032,2023-05-05 17:40:57.000,Drip Water Analysis,sodium,28.500,ppm


In [15]:
hydroponics_df_time_filtered.chemical_name.value_counts()

chemical_name
zinc                           36306
molybdenum                     36306
copper                         36306
boron                          36304
potassium                      36240
sodium                         36237
calcium                        36202
iron                           36200
phosphorus                     36200
manganese                      36200
sulphur                        36199
magnesium                      36198
bicarbonate                    36175
nitrate_n                      36168
ammonium                       36152
silicon                        36122
electrical_conductivity        36067
ph                             35963
chlorides                      18509
chloride                       17660
nitrates                       13125
sar                            11600
fluoride                         338
lead                             165
chromium                         165
arsenic                          165
cobalt                  

In [16]:
df_ = hydroponics_df_time_filtered[['analysis_name','chemical_name','unit_name']].value_counts()
df_.to_csv("output/hydroponics_unit_per_chemical.csv")

In [17]:
df_ = df_.reset_index()

In [22]:
analysis_chem_unit_df = pd.DataFrame()
for analysis in df_['analysis_name'].unique():
    print(analysis)
    chems_count_ = df_.loc[df_['analysis_name'] == analysis]
    duplicates_chem = chems_count_
    temp_analysis_chem_unit_df = pd.DataFrame()
    for index,row in duplicates_chem.iterrows():
        chemical = row['chemical_name']
        chems_count_2 = chems_count_.loc[chems_count_['chemical_name'] == chemical ]
        unit = chems_count_2.loc[chems_count_2['count'] == max(chems_count_2['count'])]['unit_name'].values[0]
        _ = pd.DataFrame({"analysis_name":[analysis],"chemical_name":[chemical], "unit_name":[unit]})
        temp_analysis_chem_unit_df = pd.concat([temp_analysis_chem_unit_df, _])
    analysis_chem_unit_df = pd.concat([analysis_chem_unit_df, temp_analysis_chem_unit_df])
analysis_chem_unit_df.to_csv("output/hydroponics_unit_per_chemical_decision.csv")

Drain Water Analysis
Drip Water Analysis
Substrate Analysis (Pumice) 1:1.5 Volume Extract
Substrate Analysis (Cocos) 1:1.5 Volume Extract
Recycle Water Analysis
Raw irrigation water analysis (for fertigation)
Fluoride Analysis
Heavy metals in Hydroponics
Fluoride Analysis (Drain)
Aluminium Analysis (Media)
Fluoride Analysis (Drip)
Fluoride Analysis (Media)
Sodium Analysis 
E coli & Total Coliforms
EC Analysis 
Aqua regia - Selected elements (ICP) Analysis
pH Analysis (Media)
Sodium Analysis
pH Analysis
Sample digestion
Bacterial test (9) in hydroponics
Biochemical Oxygen Demand
Chemical Oxygen Demand
Biuret test
Selected elements (ICP) Analysis
Alkalinity 
Turbidity
Hardness
Total Dissolved Solids
Fluoride Analysis (cocos)
Aluminium Analysis 
Fibre Analysis
Aluminium Analysis
Total Bacteria Count
pH
 EC & Sodium Analysis 
Organic Matter


PermissionError: [Errno 13] Permission denied: 'output/hydroponics_unit_per_chemical_decision.csv'

In [23]:
df = pd.DataFrame()
for index, row in analysis_chem_unit_df.iterrows():
    analysis = row['analysis_name']
    chemical = row['chemical_name']
    unit = row['unit_name']
    temp_df = hydroponics_df_time_filtered.loc[(hydroponics_df_time_filtered['analysis_name'] == analysis) & (hydroponics_df_time_filtered['chemical_name'] == chemical) & (hydroponics_df_time_filtered['unit_name'] == unit)]
    df = pd.concat([df, temp_df])

In [24]:
df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
34,522917,522917,CC003HY0107,2023-03-29 10:46:00.000,Drain Water Analysis,electrical_conductivity,2.97,mS/cm
56,520929,520929,CO006HY0294,2023-03-30 15:13:00.000,Drain Water Analysis,electrical_conductivity,2.15,mS/cm
77,522037,522037,CO006HY0292,2023-03-30 15:13:00.000,Drain Water Analysis,electrical_conductivity,2.68,mS/cm
166,526204,526204,CE361HY0019,2023-03-30 15:59:00.000,Drain Water Analysis,electrical_conductivity,1.79,mS/cm
186,526272,526272,CE361HY0017,2023-03-30 15:59:00.000,Drain Water Analysis,electrical_conductivity,0.66,mS/cm
...,...,...,...,...,...,...,...,...
548372,394241,394241,CC282HY0001,2020-07-02 09:22:00.000,Organic Matter,chloride,1130.00,ppm
548374,390409,390409,CC282HY0001,2020-07-02 09:22:00.000,Organic Matter,calcium,3.20,ppm
548375,394234,394234,CC282HY0001,2020-07-02 09:22:00.000,Organic Matter,boron,0.13,ppm
548371,394243,394243,CC282HY0001,2020-07-02 09:22:00.000,Organic Matter,bicarbonate,86.60,ppm


In [25]:
df.to_csv("output/hydroponics_analysis_cleaned.csv",chunksize=1000)

In [26]:
df.chemical_name.value_counts()

chemical_name
electrical_conductivity        102582
molybdenum                      90717
copper                          90717
zinc                            90717
boron                           90711
potassium                       90526
sodium                          90509
calcium                         90482
manganese                       90476
phosphorus                      90476
iron                            90476
sulphur                         90473
magnesium                       90473
bicarbonate                     90430
nitrate_n                       90427
ammonium                        90391
silicon                         90329
chlorides                       51968
chloride                        36754
nitrates                        17177
ph                               1149
fluoride                          524
cadmium                           241
cobalt                            241
chromium                          241
arsenic                           24